In [1]:
# gpu_embedder.py
import torch
import numpy as np
from sentence_transformers import SentenceTransformer

def make_st_model(model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = SentenceTransformer(model_name, device=device)
    # Optional: bump max length a bit if your windows are long:
    # model.max_seq_length = 384  # default 256; tradeoff: slight slowdown
    return model

def encode_texts(model: SentenceTransformer, texts: list[str], batch_size: int = 1024) -> np.ndarray:
    with torch.inference_mode():
        arr = model.encode(
            texts,
            batch_size=batch_size,
            convert_to_numpy=True,
            normalize_embeddings=True,   # cosine => inner product
            show_progress_bar=False,
        )
    return arr.astype("float32")


In [2]:
# emb_cache.py
import os, hashlib, numpy as np
from pathlib import Path

def _hash_text(t: str) -> str:
    return hashlib.md5(t.encode("utf-8")).hexdigest()

def embed_with_cache(model, texts: list[str], cache_dir: Path, batch_size: int = 1024) -> np.ndarray:
    cache_dir.mkdir(parents=True, exist_ok=True)
    vecs: list[np.ndarray] = []
    misses, miss_idx = [], []

    # 1) Load cache hits
    for i, t in enumerate(texts):
        fp = cache_dir / f"{_hash_text(t)}.npy"
        if fp.exists():
            vecs.append(np.load(fp))
        else:
            vecs.append(None)  # placeholder
            misses.append(t)
            miss_idx.append(i)

    # 2) Encode only misses
    if misses:
        from gpu_embedder import encode_texts  # local import to avoid cycles
        new_arr = encode_texts(model, misses, batch_size=batch_size)  # (m, d)
        for j, row in enumerate(new_arr):
            i = miss_idx[j]
            vecs[i] = row
            np.save(cache_dir / f"{_hash_text(texts[i])}.npy", row)

    # 3) Stack
    arr = np.vstack([v for v in vecs]).astype("float32")
    return arr


In [3]:
import time, os, hashlib, shutil
from pathlib import Path
from typing import List, Dict, Any

from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.docstore.document import Document


# --- Configuration ---
CACHE_DIR = Path(".yt_rag_cache")
CACHE_DIR.mkdir(exist_ok=True)


# --- Basic utils ---
def _vid_dir(video_id: str) -> Path:
    """Return cache subfolder for a given YouTube video."""
    vdir = CACHE_DIR / video_id
    vdir.mkdir(parents=True, exist_ok=True)
    return vdir


def _ts(sec: float) -> str:
    """Format seconds as h:mm:ss or m:ss."""
    sec = int(sec)
    h, m, s = sec // 3600, (sec % 3600) // 60, sec % 60
    return f"{h:d}:{m:02d}:{s:02d}" if h else f"{m:d}:{s:02d}"


def _hash_text(text: str) -> str:
    """Return MD5 hash of text (used for caching)."""
    return hashlib.md5(text.encode("utf-8")).hexdigest()


# --- Core helpers ---
def fetch_transcript(video_id: str, languages: List[str] = ["en"]) -> List[Dict[str, Any]]:
    """
    Fetch raw transcript segments for a YouTube video.
    Returns a list of dicts: [{'start':..., 'duration':..., 'text':...}, ...]
    """
    try:
        t0 = time.time()
        api = YouTubeTranscriptApi()
        fetched = api.fetch(video_id, languages=languages)
        data = fetched.to_raw_data()
        print(f"✅ Transcript fetched: {len(data)} segments in {time.time()-t0:.2f}s")
        return data
    except TranscriptsDisabled:
        raise RuntimeError("❌ No captions available for this video.")
    except Exception as e:
        raise RuntimeError(f"❌ Failed to fetch transcript: {e}")


def group_segments(segments: List[Dict[str, Any]],
                   target_window_s: int = 110) -> List[Dict[str, Any]]:
    """
    Greedy time-based grouping of small caption segments
    into ~target_window_s second text windows.
    """
    out, cur, start, end = [], [], None, None
    for row in segments:
        t0, t1 = row["start"], row["start"] + row.get("duration", 0)
        if start is None:
            start = t0
        end = t1
        cur.append(row["text"].strip())
        if (end - start) >= target_window_s:
            out.append({"start": start, "end": end, "text": " ".join(cur).strip()})
            cur, start, end = [], None, None
    if cur:
        out.append({"start": start, "end": end, "text": " ".join(cur).strip()})

    print(f"🧩 Windows created: {len(out)} (≈{target_window_s}s each)")
    return out


def make_docs(video_id: str, windows: List[Dict[str, Any]]) -> List[Document]:
    """
    Convert grouped transcript windows into LangChain Document objects.
    Metadata includes start/end timestamps for citation links.
    """
    docs = []
    for i, w in enumerate(windows):
        meta = {
            "video_id": video_id,
            "start": w["start"],
            "end": w["end"],
            "window_id": i,
            "time_range": f"{_ts(w['start'])}–{_ts(w['end'])}",
        }
        docs.append(Document(page_content=w["text"], metadata=meta))
    print(f"📄 Document objects: {len(docs)}")
    return docs


In [6]:
# index_build_st.py
import json, shutil, time
from pathlib import Path
import numpy as np, faiss

# reuse your existing helpers:
#  - fetch_transcript(video_id)
#  - group_segments(segments, target_window_s)
#  - make_docs(video_id, windows)
#  - _vid_dir(video_id)

EMB_CACHE_ROOT = Path(".yt_rag_cache") / "emb_cache_st"

def build_or_load_index_st(video_id: str,
                           force_rebuild: bool = False,
                           target_window_s: int = 110,
                           batch_size: int = 1024):
    vdir    = _vid_dir(video_id)
    idx_dir = vdir / "faiss_st"
    meta_fp = vdir / "meta_st.json"
    cache_dir = EMB_CACHE_ROOT / video_id

    if force_rebuild and vdir.exists():
        shutil.rmtree(vdir, ignore_errors=True)
    vdir.mkdir(parents=True, exist_ok=True)

    # Fast path: load
    idx_path = idx_dir / "index.faiss"
    metas_path = idx_dir / "metas.jsonl"
    if idx_path.exists() and meta_fp.exists() and metas_path.exists():
        index = faiss.read_index(str(idx_path))
        metas = [json.loads(l) for l in metas_path.read_text(encoding="utf-8").splitlines()]
        return index, metas

    # Build fresh
    t0 = time.time()
    segs = fetch_transcript(video_id)
    wins = group_segments(segs, target_window_s=target_window_s)
    docs = make_docs(video_id, wins)

    texts = [d.page_content for d in docs]
    model = make_st_model("sentence-transformers/all-MiniLM-L6-v2")

    t = time.time()
    arr = embed_with_cache(model, texts, cache_dir=cache_dir, batch_size=batch_size)  # (n, 384)
    t_embed = time.time() - t

    # FAISS (cosine via inner product; embeddings are normalized)
    d = arr.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(arr)

    # Save
    idx_dir.mkdir(parents=True, exist_ok=True)
    faiss.write_index(index, str(idx_path))
    with (idx_dir / "metas.jsonl").open("w", encoding="utf-8") as f:
        for doc in docs:
            f.write(json.dumps(doc.metadata, ensure_ascii=False) + "\n")
    with meta_fp.open("w", encoding="utf-8") as f:
        json.dump({
            "video_id": video_id,
            "num_windows": len(wins),
            "chunk_sec": target_window_s,
            "model": "all-MiniLM-L6-v2",
            "dim": int(d),
            "embed_time_s": round(t_embed, 3),
        }, f, ensure_ascii=False, indent=2)

    print(f"✅ Index built in {time.time()-t0:.2f}s (embed {t_embed:.2f}s) → {idx_dir}")
    return index, [doc.metadata for doc in docs]


In [7]:
VIDEO_ID = "3qHkcs3kG44"
segs = fetch_transcript(VIDEO_ID)
wins = group_segments(segs, target_window_s=110)
docs = make_docs(VIDEO_ID, wins)
print("Example:", docs[0].metadata, "\n", docs[0].page_content[:120], "...")

✅ Transcript fetched: 4076 segments in 2.50s
🧩 Windows created: 73 (≈110s each)
📄 Document objects: 73
Example: {'video_id': '3qHkcs3kG44', 'start': 0.56, 'end': 110.79899999999999, 'window_id': 0, 'time_range': '0:00–1:50'} 
 two one boom all right we're live thank you very much for doing this man i really appreciate it i've been absorbing your ...
